# Retrieve sequences from Genbank


In [1]:
from Bio import Entrez, SeqIO

VC1 ='NC_002505' # Vibrio cholerae chromosome 1
VC2 ='NC_002506' #Vibrio cholerae chromosome 2
TP = 'NC_009486' # Thermotoga petrophila 
EC = 'NC_000913' # E coli K12
EO157 = 'BA000007' # E coli O157 Sakai

In [2]:
Entrez.email='d.m.a.martin@dundee.ac.uk'
TPseqentry = next(SeqIO.parse(Entrez.efetch(db='nucleotide', id=TP, retmode='txt', rettype='gbwithparts'), 'gb'))
ECseqentry = next(SeqIO.parse(Entrez.efetch(db='nucleotide', id=EC, retmode='txt', rettype='gbwithparts'), 'gb'))
V1seqentry = next(SeqIO.parse(Entrez.efetch(db='nucleotide', id=VC1, retmode='txt', rettype='gbwithparts'), 'gb'))
V2seqentry = next(SeqIO.parse(Entrez.efetch(db='nucleotide', id=VC2, retmode='txt', rettype='gbwithparts'), 'gb'))
EO157entry = next(SeqIO.parse(Entrez.efetch(db='nucleotide', id=EO157, retmode='txt', rettype='gbwithparts'), 'gb'))



In [3]:
def reverse_complement(seq):
    '''reverse complement a sequence'''
    s2= seq[::-1].lower()
    return s2.replace('a','T').replace('c','G').replace('g','C').replace('t','A')

def get_words(seqentry, ws=13):
    seq = str(seqentry.seq)
    seq += seq[:ws]
    words = {}
    for p in range(len(seq)):
        kmer = seq[p:p+ws]
        kmer = min(kmer, reverse_complement(kmer))
        words[kmer] = words.get(kmer, 0)+1
    return [(x, words[x]) for x in sorted(words, reverse=True, key=lambda y: words[y])]


In [4]:
TPwords = get_words(TPseqentry)
V1words = get_words(V1seqentry)
V2words = get_words(V2seqentry)
ECwords = get_words(ECseqentry)
ESwords = get_words(EO157entry)


In [5]:
print('Thermatoga petrophilia', len(TPwords), len([x for x in TPwords if x[1]> 2 ]))
print('Vibrio cholerae Chr 1', len(V1words), len([x for x in V1words if x[1]> 2 ]))
print('Vibrio cholerae Chr 2', len(V2words), len([x for x in V2words if x[1]> 2 ]))
print('E coli K12', len(ECwords), len([x for x in ECwords if x[1]> 2 ]))
print('E coli O157', len(ESwords), len([x for x in ESwords if x[1]> 2 ]))   


Thermatoga petrophilia 1633320 24837
Vibrio cholerae Chr 1 2642814 34829
Vibrio cholerae Chr 2 989067 9791
E coli K12 3852770 122049
E coli O157 4344578 198332


In [6]:
TPwords[:1]

[('TACCTCTAAGGAA', 65)]

In [7]:
def compare_words(words1, words2, n=1000):
    
    w1 = [x[0] for x in words1 if x[1] >=words1[n-1][1]]
    w2 = [x[0] for x in words2 if  x[1] >= words2[n-1][1]]
    union = len(set(w1+ w2))
    overlap = len(w1)+len(w2) - union
    jacindex = overlap /union
    print('Set 1: {} Set 2: {} Union: {} Overlap: {}'.format(len(w1), len(w2), union, overlap))
    
    return jacindex

In [13]:

print('Jaccard Index: {:.03f}'.format(compare_words(V1words, V2words, n=1000)))

Set 1: 4715 Set 2: 1165 Union: 5764 Overlap: 116
Jaccard Index: 0.020
